In [1]:
import pandas as pd
from tqdm import tqdm
import re
import ast


In [2]:
# Reading the csv file
chat = pd.read_csv("chat_data.csv")

In [3]:
# To see first 5 columns more clearly since it is conversational data
pd.set_option('display.max_colwidth', None)

In [4]:
chat.head()

conversations  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          [{'from': 'human', 'value': "I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me."}\n {'from': 'gpt', 'value': "Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on?"}\n {'from': 'human', 'value': "I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It's been a really moving experience for me."}\n {'from': 'gpt', 'value': "I can understand how it can be overwhelming when we're faced with higher expectations. It's okay to acknowledge your emotions and allow yourself to feel sad in this situation. It's an important part of the healing process. What specific challenges have you been facing at work?"}\n {'from': 'human', 'value': "Well, the workload has increased significantly, and I find it hard to maintain a work-life balance. I've been staying late at the office, and it feels like I'm constantly under a pile of never-ending tasks. It's just so hard to keep up, and it's impacting my overall well-being."}\n {'from': 'gpt', 'value': "It sounds like you're dealing with a lot of pressure to perform and succeed. Remember, it's crucial to take care of yourself, both mentally and physically. A healthy work-life balance is essential. Have you tried any strategies to cope with the added workload?"}\n {'from': 'human', 'value': "I've been trying to prioritize my tasks and delegate whenever possible. I've also started practicing meditation during my breaks to help manage stress. But sometimes, it feels like no matter what I do, I can't catch a break. It's been a constant struggle."}\n {'from': 'gpt', 'value': "It's great to hear that you're already implementing some helpful strategies. Remember, progress takes time, and it's okay to have setbacks. In addition to what you're already doing, I encourage you to also communicate with your supervisor or team about your workload and discuss possible solutions together."}\n {'from': 'human', 'value': "You're right. I haven't really opened up about my struggles to my coworkers or supervisor. I guess I've been afraid of appearing weak or incapable. How can I approach this discussion without feeling vulnerable?"}\n {'from': 'gpt', 'value': "It's completely normal to feel that way, but remember, asking for support is a strength, not a weakness. Start by scheduling a conversation with your supervisor or a trusted colleague in a private and comfortable setting. Be honest about your challenges and express your willingness to find solutions together. Remember, you're not alone in this."}\n {'from': 'human', 'value': "Thank you for your understanding

# Data Processing Function

In [5]:
!python3 -m pip install tqdm

In [6]:
def process_conversation_row(conversation_str: str, conversation_id: str) -> list:
    """
    Processes a conversation string from a row to create Human/AI pair chunks.
    This version is corrected based on the clarified data structure.
    """
    # Guard Clause: If the input is not a string or is empty, skip it.
    if not isinstance(conversation_str, str) or not conversation_str.strip():
        return []

    documents = []
    try:
        # The entire string is a list, so we just replace \n with a comma
        # to make it a syntactically correct list for parsing.
        cleaned_str = conversation_str.replace('\n', ',')
        
        # Safely parse the string into a Python list
        conversation_list = ast.literal_eval(cleaned_str)
        
        # Iterate through the conversation turns to create pairs
        for i in range(0, len(conversation_list) - 1, 2):
            if conversation_list[i].get('from') == 'human' and conversation_list[i+1].get('from') == 'gpt':
                human_text = conversation_list[i].get('value', '')
                gpt_text = conversation_list[i+1].get('value', '')
                
                combined_text = f"Human: {human_text}\nAI: {gpt_text}"
                
                metadata = {
                    'conversation_id': conversation_id, # Use the ID from the 'Id' column
                    'turn_number': i 
                }
                documents.append({'text_chunk': combined_text, 'metadata': metadata})

    except (SyntaxError, ValueError) as e:
        # This will catch any rows that still have formatting issues
        print(f"Skipping row with ID '{conversation_id}' due to a parsing error: {e}")
        return []
        
    return documents

## Apply the Function and Create Your Document List

In [7]:
all_documents = []
for index, row in tqdm(chat.iterrows(), total=chat.shape[0], desc="Processing Conversations"):
    # Pass the 'conversations' string and the 'Id' to the function
    chunks = process_conversation_row(row['conversations'], row['id'])
    all_documents.extend(chunks)

print(f"\nSuccessfully processed the data.")
print(f"Total conversational chunks created: {len(all_documents)}")

# Inspect the first document to confirm it's correct
if all_documents:
    print("\n--- Example Document ---")
    print(all_documents[0])

Processing Conversations: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99086/99086 [00:08<00:00, 11118.34it/s]


Successfully processed the data.
Total conversational chunks created: 807085

--- Example Document ---
{'text_chunk': "Human: I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me.\nAI: Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on?", 'metadata': {'conversation_id': 'identity_0', 'turn_number': 0}}


## Embedding Starts and Vector Database

In [8]:
# Starting Embedding

In [9]:
from sentence_transformers import SentenceTransformer

# Load the BGE-large model.
# This is a much more powerful model and will take longer to load and use.
model_name = "BAAI/bge-large-en-v1.5"
model = SentenceTransformer(model_name)

print(f"Embedding model '{model_name}' loaded successfully.")

Embedding model 'BAAI/bge-large-en-v1.5' loaded successfully.


In [10]:
# Creation of Vector Database

In [11]:
import chromadb

# Initialize the client (this will use the same 'chroma_db' directory)
client = chromadb.PersistentClient(path="chroma_db")

# Create a NEW collection with a different name
collection_name_bge = "conversational_rag_bge"
collection = client.get_or_create_collection(name=collection_name_bge)

print(f"ChromaDB client initialized. Collection '{collection_name_bge}' is ready.")
print(f"Current document count in collection: {collection.count()}")

ChromaDB client initialized. Collection 'conversational_rag_bge' is ready.
Current document count in collection: 0


In [13]:
from sentence_transformers import SentenceTransformer
import torch # NEW: Import torch

# --- NEW: DETECT AND SET THE APPLE SILICON GPU (MPS) DEVICE ---
if torch.backends.mps.is_available():
    device = 'mps'
    print("✅ MPS (Apple Silicon GPU) is available! The model will run on the GPU.")
else:
    device = 'cpu'
    print("❌ MPS not available. Running on CPU.")
# ----------------------------------------------------------------

# Load the BGE-large model
model_name = "BAAI/bge-large-en-v1.5"
model = SentenceTransformer(model_name)

# --- NEW: MOVE THE MODEL TO THE DETECTED DEVICE (GPU) ---
model.to(device)
# -------------------------------------------------------

print(f"Embedding model '{model_name}' is now on device: '{model.device}'")

✅ MPS (Apple Silicon GPU) is available! The model will run on the GPU.
Embedding model 'BAAI/bge-large-en-v1.5' is now on device: 'mps:0'


In [14]:
# The 'all_documents' list is already prepared from our previous steps.

# Prepare the data for ChromaDB's add method
ids_to_add = [f"{doc['metadata']['conversation_id']}_turn{doc['metadata']['turn_number']}" for doc in all_documents]
documents_to_add = [doc['text_chunk'] for doc in all_documents]
metadatas_to_add = [doc['metadata'] for doc in all_documents]

# This will now run on your M4 Max GPU and be MUCH faster.
print(f"Generating embeddings with BGE-large model on device: '{device}'...")
embeddings_to_add = model.encode(documents_to_add, show_progress_bar=True, device=device).tolist()
print("Embeddings generated.")

# Add the data to the new BGE collection
collection.add(
    embeddings=embeddings_to_add,
    documents=documents_to_add,
    metadatas=metadatas_to_add,
    ids=ids_to_add
)

print(f"\nSuccessfully added {len(ids_to_add)} documents to the '{collection_name_bge}' collection.")
print(f"New document count in collection: {collection.count()}")

Generating embeddings with BGE-large model on device: 'mps'...


Batches:   0%|          | 0/25222 [00:00<?, ?it/s]

Embeddings generated.


InternalError: ValueError: Batch size of 807085 is greater than max batch size of 5461

In [15]:

# Define a batch size that is safely below the limit
batch_size = 4096

# Loop through the data in batches
for i in tqdm(range(0, len(documents_to_add), batch_size), desc="Adding documents in batches"):
    # Find the end of the batch
    end_index = i + batch_size
    
    # Get the batch data from your existing lists
    ids_batch = ids_to_add[i:end_index]
    documents_batch = documents_to_add[i:end_index]
    metadatas_batch = metadatas_to_add[i:end_index]
    embeddings_batch = embeddings_to_add[i:end_index]
    
    # Add the batch to the collection
    collection.add(
        ids=ids_batch,
        embeddings=embeddings_batch,
        documents=documents_batch,
        metadatas=metadatas_batch
    )

print("\n✅ Successfully added all documents to the collection in batches.")
print(f"Total document count in collection: {collection.count()}")

Adding documents in batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 198/198 [09:23<00:00,  2.84s/it]



✅ Successfully added all documents to the collection in batches.
Total document count in collection: 807085
